<a href="https://colab.research.google.com/github/imryoung9898/Sogang_EconAI_satellite_imagery/blob/main/2_sentinel_clip_and_download_IJ_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This code does : <br>


1.   Create cookie molds
2.   Clip reprojected Sentinel2 images 

Last modified : Jan 17 2022 by Imryoung Jeong (neptune0118@gmail.com) <br>

Recommend to work locally or switch runtime type to 'High-RAM'



# Set Up

## Install packages 
[See here](https://stackoverflow.com/questions/66811372/geopandas-and-google-colab-issue-spatial-indexes-require-either-rtree-or-pyg)

In [2]:
!pip install pandas fiona shapely pyproj rtree

     |████████████████████████████████| 15.4 MB 5.3 MB/s 
     |████████████████████████████████| 6.3 MB 55.0 MB/s 
     |████████████████████████████████| 994 kB 54.5 MB/s 


In [3]:
!pip install geopandas

     |████████████████████████████████| 1.0 MB 5.3 MB/s 


In [4]:
!pip install rioxarray

     |████████████████████████████████| 47 kB 2.4 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 19.3 MB 7.3 MB/s 
  Created wheel for rioxarray: filename=rioxarray-0.9.1-py3-none-any.whl size=54611 sha256=f70ce6ce2e8646fda796d635fa243f94e6a653b2153b4cadba42083c0c372687
  Stored in directory: /root/.cache/pip/wheels/07/da/9e/1cc57b2e7a29a206893db83e984a341e2e94378263e0798229
Successfully built rioxarray


## Import libraries

In [21]:
import rioxarray as rxr
import pandas as pd
import rasterio as rio
import geopandas as gpd
import numpy as np
import requests, zipfile, shutil, re, fiona, os, time, datetime
import rtree

from tqdm import trange, notebook
from os import listdir
from google.colab import files
from urllib.parse import unquote
from tqdm import tqdm
from zipfile import ZipFile
from os.path import basename

## Define directories & url

In [6]:
url_sentinel = "https://www.dropbox.com/s/yztv072hnuhhay9/test_sentinel_reproj.zip?dl=0"
drop_path_sentinel = "./Sentinel_reproj/"

url_tile = "https://www.dropbox.com/s/t8x7tag28phhwnj/test_tiles.zip?dl=0"
drop_path_tile = "./Cookie_mold/"

In [7]:
dir_tile_index = drop_path_tile + "test_tile_index.shp"
dir_pop_grid = drop_path_tile + "population_grid_combined_2km.shp"

# Functions

In [8]:
def download_dropbox(url, folder):
  ## version 2.0, last modified by Hyunjoo Yang (hyang@sogang.ac.kr) on Jan. 14 2022
  ## This function downloads dropbox shareable link to a local folder (tested for file downloading, but not for shared folder!)
  
  # url: dropbox shareable link for downloading
  # folder: where to download
  
  headers = {'user-agent': 'Wget/1.16 (linux-gnu)'}
  r = requests.get(url, stream=True, headers=headers)
  
  # convert dropbox shareable link ("dl=0" to "dl=1")
  url = url.replace("?dl=0", "?dl=1")

  # check if the url returns valid status code (200)
  if r.status_code == 200:
    print('The url is valid.')

    # grab filename from the url, using regular expressions (and replace space to "_" )
    file_name = unquote(re.search(r'\/([^\/]+\.([\w]+))\?dl=([01])$', url).group(1)).replace(" ", "_")

    folder_n_fname = os.path.join(folder, file_name)

    # download
    print('Begin downloading < {} >'.format(folder_n_fname))


    # Total size in bytes.
    total_size = int(r.headers.get('content-length', 0))
    block_size = 1024  # 1 Kibibyte
    t = tqdm(total=total_size, unit='B', unit_scale=True)
    with open(file_name, 'wb') as f:
      for data in r.iter_content(block_size):
        t.update(len(data))
        f.write(data)
    t.close()

    print('Finished downloading < {} >'.format(folder_n_fname))

    # unzip if zip file
    if file_name.endswith('.zip'):
      print('Extracting zip file...')
      with zipfile.ZipFile(file_name,"r") as zip_ref:
        zip_ref.extractall(folder)
    else:
      if not os.path.exists(folder):
        os.makedirs(folder)
      shutil.move(file_name, folder_n_fname)

    print('Done!')
  else:
    raise ValueError('Nothing to download: dropbox link is not valid. Check the URL link again')

In [9]:
def make_cookie_mold(sentinel_id) :


    #Create cookie molds
    guide_tile = tile_index[tile_index['sentinel_id'] == sentinel_id].copy()
    guide_tile['_merge'] = 1

    mold_wide = gpd.sjoin(pop_grid, guide_tile, how = 'left', op = 'intersects')
    mold_wide = mold_wide[mold_wide['_merge'] == 1].reset_index()
    
    ##Drop duplicated items
    mold_wide.drop_duplicates(subset = ['geometry'], keep = 'first', inplace = True)
    mold_wide.to_file("./Cookie_mold/mold_{}.shp".format(sentinel_id),
                      encoding = 'euc-kr', index = False)
    
    #Create csv that identifies which grid cells to drop
    mold_narrow = gpd.sjoin(pop_grid, guide_tile, how = 'left', op = 'within')
    mold_narrow = mold_narrow[mold_narrow['_merge'] == 1].reset_index()
    mold_narrow = mold_narrow[['id', '_merge', 'geometry']]

    csv = mold_wide.merge(mold_narrow, how = 'left', on = ['id'])
    csv.drop(columns = ['index', 'index_right', '_merge_x'], inplace = True)
    csv.rename(columns = {'_merge_y' : '_within'}, inplace = True)

    if not os.path.exists("./csv/") : os.makedirs("./csv/")
    csv.to_csv("./csv/Within_molds_{}.csv".format(sentinel_id), encoding = 'euc-kr', index = False)


In [30]:
def clip_sentinel_and_download(sentinel_id) :

    error = 0
    error_sentinel_id_list = []

    #Open reprojected images and cookie molds
    mold = fiona.open("./Cookie_mold/mold_{}.shp".format(sentinel_id), encoding = 'euc-kr')
    sentinel = rxr.open_rasterio("./Sentinel_reproj/{}.tif".format(sentinel_id)).squeeze()

    #Create output directory
    dir_sentinel_clipped = "./{}/".format(sentinel_id)
    if not os.path.exists(dir_sentinel_clipped) : os.makedirs(dir_sentinel_clipped)

    #Clip
    for i in notebook.tqdm(range(10)) :

        mask = gpd.GeoDataFrame.from_features([mold[i]]) 

        try :
            sentinel_clipped = sentinel.rio.clip(mask.geometry, drop = True, invert = False)
            sentinel_clipped.rio.to_raster(dir_sentinel_clipped + str(mask.id[0]) + ".tif")

        except :
            error += 1
            error_sentinel_id_list.append(mask.id[0])
            continue


    if error != 0 : print("{} errors occured : {}".format(error, error_sentinel_id_list))


    #Compress
    shutil.make_archive("./{}".format(sentinel_id), 'zip', dir_sentinel_clipped)

    #Download
    files.download("./{}.zip".format(sentinel_id))

# Main 

## Download zipfiles from dropbox

In [10]:
download_dropbox(url_sentinel, drop_path_sentinel)
download_dropbox(url_tile, drop_path_tile)

The url is valid.
Begin downloading < ./Sentinel_reproj/test_sentinel_reproj.zip >


100%|██████████| 744M/744M [00:12<00:00, 60.7MB/s]


Finished downloading < ./Sentinel_reproj/test_sentinel_reproj.zip >
Extracting zip file...
Done!
The url is valid.
Begin downloading < ./Cookie_mold/test_tiles.zip >


100%|██████████| 593k/593k [00:00<00:00, 5.96MB/s]

Finished downloading < ./Cookie_mold/test_tiles.zip >
Extracting zip file...
Done!


## Prepare Sentinel, tile index, population grids

In [11]:
sentinel_id_list = [f.split(".")[0] for f in os.listdir(drop_path_sentinel)]

sentinel_id_list

['52SDH,2018-06-24,0_reproj', '52SBC,2018-05-23,0_reproj']

In [12]:
tile_index = gpd.read_file(dir_tile_index) 

if tile_index.crs != 'epsg:4326':
    tile_index = tile_index.to_crs('epsg:4326')
    print('Reprojected to EPSG:4326')

tile_index['sentinel_id'] = tile_index.location.str.split("/").str[-1].str.split(".").str[0]

tile_index.head()

,location,geometry,sentinel_id
0,/Users/imryoung/Downloads/dongyoon/Sentinel_re...,"POLYGON ((125.74040 34.32421, 126.95619 34.324...","52SBC,2018-05-23,0_reproj"
1,/Users/imryoung/Downloads/dongyoon/Sentinel_re...,"POLYGON ((127.84725 38.84900, 129.11248 38.849...","52SDH,2018-06-24,0_reproj"


In [13]:
pop_grid = gpd.read_file(dir_pop_grid)

if pop_grid.crs != 'epsg:4326' :
    pop_grid = pop_grid.to_crs('epsg:4326')
    print('Reprojected to EPSG:4326')

pop_grid.head()

Reprojected to EPSG:4326


,id,r2_val,geometry
0,1,60,"POLYGON ((124.60796 37.98245, 124.63071 37.983..."
1,2,0,"POLYGON ((124.60867 37.96445, 124.63141 37.965..."
2,3,0,"POLYGON ((124.60938 37.94645, 124.63211 37.947..."
3,4,0,"POLYGON ((124.63071 37.98301, 124.65345 37.983..."
4,5,413,"POLYGON ((124.63141 37.96501, 124.65415 37.965..."


## Create cookie molds

In [14]:
for sentinel_id in sentinel_id_list :

    make_cookie_mold(sentinel_id)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.7/dist-package

## Clip and download

In [31]:
clip_sentinel_and_download(sentinel_id)

  0%|          | 0/10 [00:00<?, ?it/s]

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>